# Trait Weight Based Optimization

In [3]:
#imports required
import numpy as np
import numpy.random as rnd
import scipy
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.optimize import minimize
from numpy import linalg as LA
import cvxpy as cp

In [4]:
#installations
# !pip3 install sklearn

### Setup

We will modelling the robot team allocation to solve few independent challenges.

The species will be the 20 types of robots(divided into 4 groups).For each demonstration, each type of robot from the group will be randomly assigned to the expert.

There are 9 traits for each robot specie which are generic and mostly applicable to all of them. 

There are three task which are required to be completed with the allocation. 

In [49]:
# experiment settings
num_species = 4 #drone,rover,mini-rover,mini-drone
num_tasks = 3  #pick,search for target,move object
num_traits = 9 #speed,footprint,payload,reach,weight,sensing frequency,sensing range,color,battery capacity
num_demo = 1000

In [50]:
total_num_species = 4
rng = rnd.default_rng()
q_1 = np.concatenate([rng.normal(10,1, size=1),rng.normal(5, 1, size=1),rng.normal(7, 1, size=1),rng.normal(15, 2, size=1)])#speed
q_2 = np.concatenate([rng.normal(90,5, size=1),rng.normal(150, 9, size=1),rng.normal(120, 10, size=1),rng.normal(75, 12, size=1)]) #footprint
q_3 = np.concatenate([rng.normal(8,2, size=1),rng.normal(5, 1, size=1),rng.normal(16, 1, size=1),rng.normal(7, 2, size=1)]) #payload
q_4 = np.concatenate([rng.normal(2,1, size=1),rng.normal(3, 1, size=1),rng.normal(2, 1, size=1),rng.normal(3, 2, size=1)]) #reach
q_5 = rng.normal(np.random.randint(25,30), 1, size=total_num_species) #weight
q_6 = rng.normal(np.random.randint(210,230), 15.4, size=total_num_species) #sensing frequency
q_7 = rng.normal(np.random.randint(58,60), 0.8, size=total_num_species) #sensing range
q_8 = rnd.choice([0,1,2,3,4], total_num_species) #color
q_9 = rnd.choice(range(10,20), total_num_species) #battery
Q = np.array([q_1, q_2, q_3, q_4, q_5, q_6, q_7, q_8, q_9]).T
Q

array([[  8.41108973,  90.36590457,   5.78922572,   2.18786945,
         29.07967107, 221.50752109,  59.37807416,   1.        ,
         14.        ],
       [  4.10817297, 149.06086455,   5.90908126,   4.12432975,
         27.94674664, 219.12846354,  58.10482788,   0.        ,
         11.        ],
       [  7.10151428, 127.89401355,  15.12723067,   1.24708419,
         27.36687847, 203.00186519,  58.15604231,   1.        ,
         16.        ],
       [ 11.56845354,  79.71122655,   8.62953763,   8.1132141 ,
         28.71544159, 243.96353961,  59.5151287 ,   0.        ,
         12.        ]])

In [51]:
# total_num_species = 20

# rng = rnd.default_rng()
# q_1 = np.concatenate([rng.normal(10,1, size=5),rng.normal(5, 1, size=5),rng.normal(7, 1, size=5),rng.normal(15, 2, size=5)])#speed
# q_2 = np.concatenate([rng.normal(90,5, size=5),rng.normal(150, 9, size=5),rng.normal(120, 10, size=5),rng.normal(75, 12, size=5)]) #footprint
# q_3 = np.concatenate([rng.normal(8,2, size=5),rng.normal(5, 1, size=5),rng.normal(16, 1, size=5),rng.normal(7, 2, size=5)]) #payload
# q_4 = np.concatenate([rng.normal(2,1, size=5),rng.normal(3, 1, size=5),rng.normal(2, 1, size=5),rng.normal(3, 2, size=5)]) #reach
# q_5 = rng.normal(np.random.randint(25,30), 1, size=total_num_species) #weight
# q_6 = rng.normal(np.random.randint(210,230), 15.4, size=total_num_species) #sensing frequency
# q_7 = rng.normal(np.random.randint(58,60), 0.8, size=total_num_species) #sensing range
# q_8 = rnd.choice([0,1,2,3,4], total_num_species) #color
# q_9 = rnd.choice(range(10,20), total_num_species) #battery
# Q_master = np.array([q_1, q_2, q_3, q_4, q_5, q_6, q_7, q_8, q_9])

# a = rnd.randint(0,5)
# b = rnd.randint(5,10)
# c = rnd.randint(10,15)
# d = rnd.randint(15,20)
# indx = np.array([a,b,c,d])
# print(indx)
# Q = Q_master[:, indx].T
# Q

In [52]:
# np.set_printoptions(precision=2)
# print(Q_master.shape)
# Q_master

In [53]:
#speed,footprint,payload,reach,weight,sensing frequency,sensing range,color,battery capacity
y1 = [1,0,1,1,0,1,1,0,0]#pick
y2 = [0,1,0,0,0,1,1,0,1]#search for target
y3 = [1,1,1,1,1,1,0,0,1]#move object
y_star = np.array([y1,y2,y3])
y_star

array([[1, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 1]])

In [54]:
def minimize_func(input_x):
    x = input_x.reshape((3,4))
    diff = (y_star - x@Q)
    return LA.norm(diff,2)

In [55]:
x = np.random.randint(0,2,size = (3,4))
in_x = x.flatten()
x

array([[0, 0, 0, 0],
       [0, 1, 1, 0],
       [1, 1, 1, 0]])

In [56]:
minimize_func(in_x)

926.3212467020619

In [61]:
res = minimize(minimize_func, in_x, method='Powell', tol=1e-6)
# print(res)
X_app = np.array(res.x).reshape((3,4))
# X_app[X_app<0] = 0
X_app

array([[ 2.18141819e-02,  1.63422440e-07, -4.64204219e-08,
         1.19426243e-10],
       [-2.42518896e+00, -1.62231693e-01,  1.18357051e+00,
         1.30508550e+00],
       [-2.45918863e+00, -5.28585080e-01,  1.28641752e+00,
         1.71841150e+00]])

In [64]:
print(X_app.dot(Q))
Y = X_app.dot(Q)
print(LA.norm(y_star-Y))

Y2 = abs(X_app).dot(Q)
LA.norm(y_star-Y2)



[[  0.18348138   1.97127671   0.12628749   0.0477272    0.63435254
    4.83203178   1.29529092   0.02181414   0.3053996 ]
 [  2.43800604  12.06475802  14.16782195   6.08935632  -5.19080856
  -14.0868775   -6.92538648  -1.24161845  -1.13903993]
 [  6.15890073   0.48363314  16.92878092   7.98566848  -1.73445309
   19.8180919    0.34920274  -1.17277112   0.96054148]]
37.46546779097942


1992.0933974638913

In [59]:
X = cvxpy.Variable((num_tasks, num_species), integer=True)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

NameError: name 'cvxpy' is not defined

array([2., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.])
